In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [5]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [3]:
SELECT Me.NomeMedicamento, COUNT(*) NUM FROM Medicamento Me, Pessoa Pe, UsoMedicamento Mu WHERE Me.Codigo = Mu.CodMedicamento AND Pe.Id = Mu.IdPessoa GROUP BY Me.NomeMedicamento;

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [10]:
DROP VIEW IF EXISTS Contagem_Medicamento;
CREATE VIEW Contagem_Medicamento AS
SELECT Me.Codigo, COUNT(*) NUM FROM Medicamento Me, Pessoa Pe, UsoMedicamento Mu 
WHERE Me.Codigo = Mu.CodMedicamento AND Pe.Id = Mu.IdPessoa GROUP BY Me.NomeMedicamento;

SELECT Codigo, NUM FROM Contagem_Medicamento ORDER BY NUM DESC LIMIT 1;

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [12]:
SELECT CodMedicamento, COUNT(*) NUM FROM UsoMedicamento
GROUP BY CodMedicamento ORDER BY NUM DESC LIMIT 1;

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [13]:
DROP VIEW IF EXISTS Contagem_Medicamento;
CREATE VIEW Contagem_Medicamento AS
SELECT Me.Codigo, COUNT(*) NUM FROM Medicamento Me, Pessoa Pe, UsoMedicamento Mu 
WHERE Me.Codigo = Mu.CodMedicamento AND Pe.Id = Mu.IdPessoa GROUP BY Me.NomeMedicamento;

SELECT Me.NomeMedicamento FROM Contagem_Medicamento Cm, Medicamento Me WHERE Me.Codigo = Cm.Codigo ORDER BY NUM DESC LIMIT 1;

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [16]:
DROP VIEW IF EXISTS Contagem_Media_Medicamentos;
CREATE VIEW Contagem_Media_Medicamentos AS
SELECT COUNT(*) NUM FROM UsoMedicamento GROUP BY IdPessoa;

SELECT AVG(NUM) FROM Contagem_Media_Medicamentos;

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [18]:
DROP VIEW IF EXISTS Contagem_Media_Medicamentos;
CREATE VIEW Contagem_Media_Medicamentos AS
SELECT COUNT(*) NUM FROM UsoMedicamento GROUP BY IdPessoa;

SELECT IdPessoa, COUNT(*) NUM FROM UsoMedicamento GROUP BY IdPessoa HAVING NUM > (SELECT AVG(NUM) FROM Contagem_Media_Medicamentos);

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [29]:
SELECT Pe.Idade, COUNT(*) NUM FROM Pessoa Pe, UsoMedicamento Um 
WHERE Pe.Id = Um.IdPessoa GROUP BY Pe.Idade ORDER BY Pe.Idade DESC;

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [ ]:
DROP VIEW UsoMedicamentoB IF EXISTS;
DROP VIEW MedicamentosConjunto IF EXISTS;

CREATE VIEW UsoMedicamentoB AS
SELECT IdPessoa IdPessoaB, CodMedicamento CodMedicamentoB
FROM UsoMedicamento;

CREATE VIEW MedicamentosConjunto AS
SELECT U1.CodMedicamento medicamentoA, U2.CodMedicamentoB medicamentoB
FROM UsoMedicamento U1, UsoMedicamentoB U2
WHERE U1.IdPessoa = U2.IdPessoaB AND U1.CodMedicamento > U2.CodMedicamentoB;

SELECT * FROM MedicamentosConjunto;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [ ]:
SELECT medicamentoA, medicamentoB, COUNT(*) QuantidadeConjunta
FROM MedicamentosConjunto
GROUP BY medicamentoA, medicamentoB;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [30]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [31]:
UPDATE Matriz
SET IdadeAte60 = TRUE
WHERE Idade <=60;

SELECT * FROM Matriz;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [32]:
SELECT DISTINCT IdPessoa
             FROM UsoMedicamento
             WHERE DiasUso > 365;

UPDATE Matriz
SET MaisUmAnoMedicamento = TRUE
WHERE Id IN (SELECT DISTINCT IdPessoa
             FROM UsoMedicamento
             WHERE DiasUso > 365);
             
SELECT * FROM Matriz;

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [33]:
UPDATE Matriz
SET MedicamentosAcimaMedia = TRUE
WHERE Id IN (
    SELECT IdPessoa
    FROM UsoMedicamento
    GROUP BY IdPessoa
    HAVING COUNT(*) > (
        SELECT AVG(NumeroMedicamentos) MediaMedicamentos
        FROM (SELECT COUNT(*) Numeromedicamentos
              FROM UsoMedicamento
              GROUP BY IdPessoa)));
              
SELECT * FROM Matriz WHERE MedicamentosAcimaMedia = TRUE;

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

In [37]:
--Após a Matriz criada, proponho fazer uma análise sobre qual dos gêneros tem mais pacientes tomando uma quantidade 
--de medicamentos acima da média

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [50]:
DROP VIEW IF EXISTS Contagem_Acima_Media_Medicamentos;
CREATE VIEW Contagem_Acima_Media_Medicamentos AS
SELECT Genero, COUNT(*) MEDICAMENTOS_ACIMA_DA_MEDIA FROM Matriz WHERE MedicamentosAcimaMedia = TRUE GROUP BY Genero;

DROP VIEW IF EXISTS Numero_Pessoas;
CREATE VIEW Numero_Pessoas AS
SELECT Genero, COUNT(*) NUM_PESSOAS FROM Matriz GROUP BY Genero;

SELECT Np.Genero, Np.NUM_PESSOAS, Ca.MEDICAMENTOS_ACIMA_DA_MEDIA FROM Numero_Pessoas Np, Contagem_Acima_Media_Medicamentos Ca WHERE Np.Genero = Ca.Genero;